In [1]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

import chromedriver_autoinstaller
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.get("https://www.naver.com")
time.sleep(2)

In [3]:
# 영화배우 이름 검색
name = '김혜수'

In [4]:
# 네이버 검색어 입력 후 검색
selector = "#query"
element = driver.find_element(By.CSS_SELECTOR, selector)
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거
element.send_keys(name)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(1)

In [5]:
# "수상" 클릭
driver.find_element(By.LINK_TEXT, "수상").click()
time.sleep(1)

In [6]:
# 수상 갯수 세기
selector = ".this_text_number"
prize_cnt = int(driver.find_element(By.CSS_SELECTOR, selector).text)
prize_cnt

48

In [7]:
# 1page의 첫번째 수상내역 텍스트 확인
selector = "#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > \
div:nth-child(1) > dl > div:nth-child(1) > dd > ul > li > span"
driver.find_element(By.CSS_SELECTOR, selector).text

'제28회 춘사국제영화제 여우주연상'

In [8]:
# 반복문으로 1page 수상내역 전체 텍스트 수집 후 리스트로 저장
text_list = []

for i in range(1, 9):  # 한 페이지당 최대 8개 
    selector = f"#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > \
    div:nth-child(1) > dl > div:nth-child({i}) > dd > ul > li > span"
    prize_text = driver.find_element(By.CSS_SELECTOR, selector).text
    text_list.append(prize_text)
    # print(prize_text)
text_list

['제28회 춘사국제영화제 여우주연상',
 '제44회 청룡영화상 청룡영화상',
 '제7회 금융의날 저축투자부문 대통령표창',
 '제2회 아시아콘텐츠어워즈 특별상',
 '제55회 백상예술대상 바자 아이콘상',
 '제10회 올해의 영화상 심사위원상',
 '제1회 tvN10 어워즈 여자배우상',
 '제52회 백상예술대상 TV부문 여자 최우수연기상']

In [94]:
# 수상내역 1page 첫번째 연도 확인
selector = "#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > div:nth-child(1) > dl > div:nth-child(1) > dt"
driver.find_element(By.CSS_SELECTOR, selector).text

'2023'

In [95]:
# 반복문으로 1page 안의 전체 연도 수집 후 리스트로 저장
year_list = []

for i in range(1, 9):
    selector = f"#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > \
    div:nth-child(1) > dl > div:nth-child({i}) > dt"
    year = driver.find_element(By.CSS_SELECTOR, selector).text
    year_list.append(year)
    # print(year)
year_list

['2023', '2023', '2022', '2020', '2019', '2019', '2016', '2016']

In [96]:
# 데이터프레임으로 저장
pd.DataFrame({'prize':text_list, 'year':year_list})

,prize,year
0,제28회 춘사국제영화제 여우주연상,2023
1,제44회 청룡영화상 청룡영화상,2023
2,제7회 금융의날 저축투자부문 대통령표창,2022
3,제2회 아시아콘텐츠어워즈 특별상,2020
4,제55회 백상예술대상 바자 아이콘상,2019
5,제10회 올해의 영화상 심사위원상,2019
6,제1회 tvN10 어워즈 여자배우상,2016
7,제52회 백상예술대상 TV부문 여자 최우수연기상,2016


### page 바꾸면서 수상내역 크롤링

In [97]:
# # 다음 페이지로 이동 클릭
# selector = ".pg_next.on"
# driver.find_element(By.CSS_SELECTOR, selector).click()

In [ ]:
# 1page에서 데이터 크롤링
# 다음 페이지 이동 클릭
# 2page에서 데이터 크롤링
# ~ 마지막 페이지까지


In [9]:
# 총 페이지 번호 확인
selector = "._total"
page_num = int(driver.find_element(By.CSS_SELECTOR, selector).text)
page_num

6

In [48]:
# 이중 for문

# try, except 구문(예외 처리) 
# - try 밑에를 실행하고, 에러나면 except 밑에를 실행하세요.

# break : 현재 반복문을 빠져나가세요.

In [10]:
# 페이지를 이동하면서 각 페이지의 수상내역을 수집 후 리스트로 저장하는 코드 
# 예외처리, 이중 for문, break 

text_list = []

for j in range(1, page_num+1):   # j는 페이지(j=1,2,3,4,5) 
    try:
        for i in range(1, 9):  # i는 수상내역, 한 페이지당 최대 8개 
            try:  # 다음을 실행해, 에러나면 except 밑에 것을 실행해.
                selector = f"#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > \
                div:nth-child({j}) > dl > div:nth-child({i}) > dd > ul > li > span"
                prize_text = driver.find_element(By.CSS_SELECTOR, selector).text
                text_list.append(prize_text)
                # print(prize_text)
            except:
                break           
        # 다음 페이지로 이동 클릭
        selector = ".pg_next.on"
        driver.find_element(By.CSS_SELECTOR, selector).click()
        time.sleep(2)
    except:
        break
print(len(text_list))
text_list

48


['제28회 춘사국제영화제 여우주연상',
 '제44회 청룡영화상 청룡영화상',
 '제7회 금융의날 저축투자부문 대통령표창',
 '제2회 아시아콘텐츠어워즈 특별상',
 '제55회 백상예술대상 바자 아이콘상',
 '제10회 올해의 영화상 심사위원상',
 '제1회 tvN10 어워즈 여자배우상',
 '제52회 백상예술대상 TV부문 여자 최우수연기상',
 '제21회 춘사영화상 여우주연상',
 '제2회 한국영화제작가협회상 여우주연상',
 '제35회 한국영화평론가협회상 여우주연상',
 '제35회 황금촬영상 시상식 최우수 여우주연상',
 'APN 아시아를 빛낸 영화인상',
 'KBS 연기대상 대상',
 'KBS 연기대상 베스트 커플상',
 '제21회 대한민국 문화연예대상 드라마부문 대상',
 '제20회 대한민국 문화연예대상 영화부문 최우수연기상',
 '제12회 대한민국영상대전 포토제닉상 영화배우부문',
 '제32회 청룡영화상 인기스타상',
 'SBS 연기대상 10대 스타상',
 '제2회 스타일 아이콘 어워즈 여자 탤런트부문상',
 '제2회 스타일 아이콘 어워즈 아이콘 오브 더 이어상',
 '제1회 대한민국영화연기대상 여우주연상',
 '제8회 대한민국 영상대전 포토제닉상',
 '제2회 피어선 영화제 뉴 커런츠 무비스타 여자 최우수상',
 '제2회 대한민국 대학영화제 올해의 여자 배우상',
 '제27회 청룡영화상 인기상',
 '제27회 청룡영화상 여우주연상',
 '제14회 춘사대상영화제 인기상',
 '제14회 춘사대상영화제 여우주연상',
 '코리아 베스트 드레서상 영화부문상',
 '동아TV 패션뷰티상 패션아이콘상',
 '제42회 대종상영화제 여우주연상',
 '제41회 백상예술대상 최우수 여자 연기상',
 'MBC연기대상 최우수상',
 'KBS 연기대상 대상',
 '한국주얼리페어 베스트 주얼리 레이디',
 '한국패션사진가협회 포토제닉 연기자상',
 'MBC 연기대상 최우수상',
 '백상예술대상 여자 연기상',
 'MBC 연기대상 대상',
 'MBC 연기대상 최우수상',
 '백상예

In [60]:
# print(len(text_list))
# text_list

In [ ]:
# [실습1] 네이버 영화배우 수상내역 크롤링
# 페이지 이동하면서 연도 수집 후 리스트에 저장
# 데이터프레임, csv로 저장

# [실습1-2] 다른 인물로 수상내역 크롤링

# [실습2] 프로필, 경력사항 크롤링

In [103]:
# [실습1-1] 네이버 영화배우 수상내역 크롤링
# 페이지 이동하면서 연도 수집 후 리스트에 저장
# 데이터프레임, csv로 저장

# 페이지를 이동하면서 각 페이지의 수상내역을 수집 후 리스트로 저장하는 코드 
# 예외처리, 이중 for문, break 

year_list = []

for j in range(1, page_num+1):   # j는 페이지(j=1,2,3,4,5) 
    try:
        for i in range(1, 9):  # i는 연도, 한 페이지당 최대 8개 
            try:  # 다음을 실행해, 에러나면 except 밑에 것을 실행해.
                selector = f"#main_pack > section > div.cm_content_wrap._cm_content_area_prize_tab > div > div > div._panel_wrapper > \
                div:nth-child({j}) > dl > div:nth-child({i}) > dt"
                prize_year = int(driver.find_element(By.CSS_SELECTOR, selector).text)
                year_list.append(prize_year)
                # print(year_list)
            except:
                break           
        # 다음 페이지로 이동 클릭
        selector = ".pg_next.on"
        driver.find_element(By.CSS_SELECTOR, selector).click()
        time.sleep(2)
    except:
        break

print(len(year_list))
year_list

48


[2023,
 2023,
 2022,
 2020,
 2019,
 2019,
 2016,
 2016,
 2016,
 2015,
 2015,
 2015,
 2015,
 2013,
 2013,
 2013,
 2012,
 2011,
 2011,
 2009,
 2009,
 2009,
 2007,
 2007,
 2006,
 2006,
 2006,
 2006,
 2006,
 2006,
 2006,
 2005,
 2005,
 2005,
 2004,
 2003,
 2003,
 2000,
 1999,
 1996,
 1996,
 1995,
 1995,
 1995,
 1993,
 1991,
 1987,
 1986]

In [104]:
# 데이터프레임으로 저장
df_prize_year = pd.DataFrame({'prize':text_list, 'year':year_list})
df_prize_year

,prize,year
0,제28회 춘사국제영화제 여우주연상,2023
1,제44회 청룡영화상 청룡영화상,2023
2,제7회 금융의날 저축투자부문 대통령표창,2022
3,제2회 아시아콘텐츠어워즈 특별상,2020
4,제55회 백상예술대상 바자 아이콘상,2019
5,제10회 올해의 영화상 심사위원상,2019
6,제1회 tvN10 어워즈 여자배우상,2016
7,제52회 백상예술대상 TV부문 여자 최우수연기상,2016
8,제21회 춘사영화상 여우주연상,2016
9,제2회 한국영화제작가협회상 여우주연상,2015


In [107]:
df_prize_year['year'].min()

1986

In [1]:
# 이전 페이지로 돌아가기
driver.back()

NameError: name 'driver' is not defined